In [ ]:
!nvidia-smi


Thu Nov 23 00:25:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers==4.28.0 datasets nltk
# !pip install --upgrade accelerate
!pip install -q bitsandbytes accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel .

In [ ]:
from transformers import pipeline, set_seed, GPT2LMHeadModel, AutoTokenizer, TextDataset, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, EarlyStoppingCallback, BitsAndBytesConfig, GPTQConfig
import datasets
import numpy as np
import math
from datasets import load_from_disk
from peft import LoraConfig, get_peft_model
import torch
import torch.nn as nn
import bitsandbytes as bnb
fixedTexts= ['<|beginofdes|>','<|endofdes|>','<br>', '\n','\r','TIKI','Ảnh']

In [ ]:
model = AutoModelForCausalLM.from_pretrained("imthanhlv/vigpt2medium")
tokenizer = AutoTokenizer.from_pretrained("imthanhlv/vigpt2medium")
special_tokens_dict = {'additional_special_tokens': ['<|beginofdes|>','<|endofdes|>', '<br>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/781k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/439k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [ ]:
tokenizer

GPT2TokenizerFast(name_or_path='imthanhlv/vigpt2medium', vocab_size=50257, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endofdes|>', '<|beginofdes|>', '<br>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50257: AddedToken("<|endo

In [ ]:
text = "<|beginofdes|>Hiếu Trương Công<|beginofdes|>làm<|endofdes|>đồ án Trí tuệ nhân tạo"

In [ ]:
tokenizer.encode(text)


[50259, 13692, 2422, 738, 50259, 5217, 50258, 12851, 714, 3441, 3784, 555, 840]

In [ ]:
tokenizer.decode([50258, 13692, 2422, 738, 50258, 5217, 50259, 12851, 714, 3441, 3784, 555, 840])


'<|endofdes|>Hiếu Trương Công<|endofdes|>làm<|beginofdes|>đồ án Trí tuệ nhân tạo'

In [ ]:
def promptFinalize(textPrompt):
  newArray=[]
  arrays = textPrompt.split('.')
  for i in range(textPrompt.count('.')):
    newArray.append(arrays[i])
  outcome='.'.join(newArray)+'.'
  return outcome


In [ ]:
def get_text_ds(path):
    lines = []
    text = ''
    with open(path) as f:
        for line in f:
            if line.find('xxxxxEndxxxxx') != -1:
                if text.strip():
                    lines.append(text.strip())
                    text = ''
                else:
                    continue
            else:
                text = text + line
    vi_ds = datasets.Dataset.from_dict({'text': lines})
    return vi_ds

In [ ]:
train_ds = get_text_ds('dataset_train.txt')
test_ds = get_text_ds('dataset_test.txt')


In [ ]:
train_ds


Dataset({
    features: ['text'],
    num_rows: 571
})

In [ ]:
test_ds


Dataset({
    features: ['text'],
    num_rows: 144
})

In [ ]:
context_length = 512
def tokenize(batch):
  outputs = tokenizer(
      batch['text'],
      truncation=True,
      max_length=context_length,
      return_overflowing_tokens=True,
      return_length=True
  )
  input_batch = []
  for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
      input_batch.append(input_ids)
  result = {}
  result['input_ids'] = input_batch
  return result


tokenized_train_datasets = train_ds.map(
    tokenize, batched=True, remove_columns=['text']
)

Map:   0%|          | 0/571 [00:00<?, ? examples/s]

In [ ]:
tokenized_test_datasets = test_ds.map(
    tokenize, batched=True, remove_columns=['text']
)

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

In [ ]:
# eval_bs_ds = get_text_ds('dataset_eval_bs.txt')
# tokenized_evalbs_datasets = eval_bs_ds.map(
#     tokenize, batched=True, remove_columns=['text']
# )

Map:   0%|          | 0/116 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_datasets

Dataset({
    features: ['input_ids'],
    num_rows: 571
})

In [ ]:
tokenized_test_datasets


Dataset({
    features: ['input_ids'],
    num_rows: 144
})

In [ ]:
tokenizer.decode(tokenized_train_datasets[0]['input_ids'])


'Hiếu Trương Công<|beginofdes|>là một người đẹp trai vừa tài năng của Việt Nam trong những năm đầu thế kỷ 21 và cả sau này. Hiếu có những thành công vang dội là trở thành top 1 server.'

In [ ]:
tokenizer.decode(tokenized_test_datasets[0]['input_ids'])


'trang phục<|beginofdes|>trang phục với họa tiết hoa là item cần có trong tủ quần áo ngày hè của phái nữ. tuy nhiên, xu hướng này đều thay đổi theo từng năm. mùa hè năm nay, những chiếc đầm hoa tay bồng hoặc cổ chữ v chính là lựa chọn số 1 của các mỹ nhân việt cho những chuyến du lịch hoặc kết hợp với streetstyle, phong cách hàng ngày.<|endofdes|>'

In [ ]:
train_data_path = 'article_512_data/train'
tokenized_train_datasets.save_to_disk(train_data_path)

test_data_path = 'article_512_data/test'
tokenized_test_datasets.save_to_disk(test_data_path)

Saving the dataset (0/1 shards):   0%|          | 0/571 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/144 [00:00<?, ? examples/s]

In [ ]:
# Load dataset
train_dataset = load_from_disk("article_512_data/train")
test_dataset = load_from_disk("article_512_data/test")
torch.set_default_dtype(torch.float32)

compute_dtype = getattr(torch, "float")
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_quant_type="nf8",
    bnb_8bit_compute_dtype=compute_dtype,
    bnb_8bit_use_double_quant=False,
)
model = AutoModelForCausalLM.from_pretrained("imthanhlv/vigpt2medium")
tokenizer = AutoTokenizer.from_pretrained("imthanhlv/vigpt2medium")
special_tokens_dict = {'additional_special_tokens': ['<|beginofdes|>','<|endofdes|>', '<br>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
tokenizer.pad_token = tokenizer.eos_token
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)
config = LoraConfig(
    r=3000,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="all",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)
# Data collator for padding
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors="pt")

batch_size = 6
gradient_accumulation_steps = 2
num_gpu = 8

step_per_batch = math.ceil(len(train_dataset) / (batch_size * gradient_accumulation_steps * num_gpu))
print('step_per_batch', step_per_batch)

def train(model):

    training_args = TrainingArguments(
        output_dir='training_article',
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        evaluation_strategy="steps",
        num_train_epochs=5,
        save_steps=step_per_batch,
        eval_steps=step_per_batch,
        logging_steps=step_per_batch,
        learning_rate=1e-5,
        warmup_steps=step_per_batch * 5,
        save_total_limit=3,
        load_best_model_at_end=True,
        prediction_loss_only=True,
        metric_for_best_model='loss',
    )

    trainer = Trainer(
        model=model,
        data_collator=data_collator,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        callbacks=[EarlyStoppingCallback(early_stopping_patience = 4)]
    )
    trainer.place_model_on_device = False
    trainer.train()
    trainer.save_model("training_article/best_model")
    tokenizer.save_pretrained("training_article/best_model")

train(model)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/model.py:302: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


step_per_batch 6


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
6,36.540100,24.728456
12,34.987500,24.330803
18,35.438400,23.651567
24,35.346900,22.691078
30,32.910200,21.442225
36,31.391700,19.997894
42,30.182700,18.549543
48,27.744700,17.103825
54,26.188500,15.654653
60,24.945000,14.209091


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [ ]:
MODEL_NAME = './training_article/best_model'
# MODEL_NAME = './training_article/checkpoint-162'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
textPrompt=''
def generate(model, input_ids, max_length = 100, k = 3, on = 3):
  generated_sequence = model.generate(
      input_ids = input_ids,
      max_new_tokens=max_length,
      num_beams=3,
      no_repeat_ngram_size=3,
      num_return_sequences=1,
      eos_token_id=50259,
      pad_token_id=50259,
      use_cache=True
      )
  x=generated_sequence[0]
  text = tokenizer.decode(x, skip_special_tokens=False)
  for y in fixedTexts:
    printText = text.replace(y, ' ')
  arrays = text.split('\n')
  text=promptFinalize(' '.join(arrays))
  return text

def printPrompt(prompt):
  input_ids = tokenizer('{}'.format(prompt), return_tensors='pt').input_ids
  text=generate(model, input_ids,100,3,1)
  return text
# textPrompt=printPrompt('áo thun')
# textPrompt=printPrompt('áo len')
# textPrompt=printPrompt('áo sơ mi')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def removeCoincideItem(input_array):
  unique_items = {}
  result_array = []
  for item in input_array:
      item_value = item['item']
      if item_value not in unique_items:
          result_array.append(item)
          unique_items[item_value] = True

  return result_array

total_test_dataset = []
for i in range(len(tokenized_test_datasets)):
  text = tokenizer.decode(tokenized_test_datasets[i]['input_ids'])
  listSplit = text.replace('<|endofdes|>','').split('<|beginofdes|>')
  total_test_dataset.append({"item": listSplit[0], "value": listSplit[1]})

total_test_dataset = removeCoincideItem(total_test_dataset)
len(total_test_dataset)

116

In [ ]:
f1ScoreList=[]

def get_short_sentence(sentence1, sentence2):
    short, long = min(sentence1, sentence2), max(sentence1, sentence2)
    return {"short": short, "long": long}

def f1_score(sentence1, sentence2):
  words1 = sentence1.split()
  words2 = sentence2.split()
  matches = 0
  for word in words1:
    if word in words2:
      matches += 1
  precision = matches / len(words2)
  recall = matches / len(words1)
  if (precision == 0 and recall == 0): return 0
  result = 2 * precision * recall / (precision + recall)
  if (result > 1): return 1
  return result
for i in range(len(total_test_dataset)):
  print(i)
  item = total_test_dataset[i]["item"]
  testPrompt = total_test_dataset[i]["value"]
  predictions = printPrompt("{}".format(item))
  sentences = get_short_sentence(predictions, testPrompt)
  shortSen = sentences['short']
  longSen = sentences['long']
  f1_scoreCalculate = f1_score(shortSen, longSen)
  f1ScoreList.append(f1_scoreCalculate)

# In ra kết quả
def calculate_average(f1ScoreList):
  sum = 0
  for number in f1ScoreList:
    sum += number
  return sum / len(f1ScoreList)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115


In [ ]:
average = calculate_average(f1ScoreList)
[print(i) for i in f1ScoreList]
print('\n')
print('Tổng các mẫu được dự đoán: {} mẫu'.format(len(total_test_dataset)))
print('\n')
print('F1 score: {}\n'.format(average))

0.1843971631205674
0.07643312101910828
0.3120567375886525
0.14893617021276595
0.32499999999999996
0.189873417721519
0.101010101010101
0.29113924050632906
0.11764705882352941
0.10218978102189782
0.3439490445859873
0.6419753086419753
0.42553191489361697
0.08163265306122448
0.4817518248175182
0.3870967741935484
0.3448275862068966
0.16049382716049385
0.7752808988764045
0.6826347305389221
0.7368421052631577
0.3414634146341463
0.3364485981308411
0.7346938775510204
0.5
0.33121019108280253
0.2891566265060241
0.33986928104575165
0.3055555555555556
0.430379746835443
0.618421052631579
0.5185185185185185
0.5
0.4473684210526316
0.6438356164383562
0.7037037037037037
0.19999999999999998
0.5962732919254657
0.4113475177304965
0.4635761589403974
0.2184873949579832
0.43373493975903615
0.41916167664670656
0.6206896551724138
0.6265060240963854
0.5818181818181818
0.5529411764705883
0.5681818181818183
0.4539877300613497
0.34730538922155685
0.6625766871165644
0.2289156626506024
0.670807453416149
0.56296296296